In [4]:
import pandas as pd
import numpy as np

In [66]:
df = pd.read_csv("../data/season_history/nfl_season_history.csv")
df = df.rename({"Pts":"pts_winner", "Pts.1":"pts_loser"}, axis=1)

df_bears = df[df["Winner/tie"].eq("Chicago Bears") | df["Loser/tie"].eq("Chicago Bears")]

df_bears["bears_score"] = np.where(df_bears["Winner/tie"].eq("Chicago Bears"), df_bears.pts_winner, df_bears.pts_loser) 
df_bears["opp_score"] = np.where(df_bears["Winner/tie"].eq("Chicago Bears"), df_bears.pts_loser, df_bears.pts_winner) 
df_bears["opp"] = np.where(df_bears["Winner/tie"].eq("Chicago Bears"), df_bears["Loser/tie"], df_bears["Winner/tie"]) 
df_bears["score_diff"] = df_bears.bears_score - df_bears.opp_score
df_bears["result"] = np.where(df_bears.score_diff <0, "L", "W")
df_bears["countit"] = 1
df_piv = df_bears.pivot_table(index="season_year", columns="result", values="countit", aggfunc="sum")
df_piv["total"] = df_piv.W + df_piv.L
df_piv["win_percentage"] = df_piv.W / df_piv.total

/tmp/ipykernel_658789/51190907.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_bears["bears_score"] = np.where(df_bears["Winner/tie"].eq("Chicago Bears"), df_bears.pts_winner, df_bears.pts_loser)
/tmp/ipykernel_658789/51190907.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_bears["opp_score"] = np.where(df_bears["Winner/tie"].eq("Chicago Bears"), df_bears.pts_loser, df_bears.pts_winner)
/tmp/ipykernel_658789/51190907.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a sli

In [75]:
chart_title = "Chicago Bears Winning Percentage 1970-2021"
x_cfgs = {"shorthand":"season_year:O", "title":"Season Year"}
y_cfgs = {"shorthand":"win_percentage", "title":"Win %"}
# background etc.
primary_color = "#0B162A" 
# font etc.
secondary_color = "#c83803"
axisLeftKwargs = dict(labelColor=secondary_color, titleColor=secondary_color, labelFontSize=15, titleFontSize=17)
axisBottomKwargs = dict(labelColor=secondary_color, titleColor=secondary_color,labelFontSize=15, titleFontSize=17)
legendKwargs = dict(labelColor=secondary_color, titleColor=secondary_color, strokeColor='gray', fillColor=primary_color, padding=10, cornerRadius=10,orient='top-right', labelFontSize=15, titleFontSize=15)

In [84]:
df_piv.reset_index().query("season_year > 1989")

result,season_year,L,W,total,win_percentage
20,1990,6,12,18,0.666667
21,1991,6,11,17,0.647059
22,1992,11,5,16,0.312500
23,1993,9,7,16,0.437500
24,1994,8,10,18,0.555556
25,1995,7,9,16,0.562500
26,1996,9,7,16,0.437500
27,1997,12,4,16,0.250000
28,1998,12,4,16,0.250000
29,1999,10,6,16,0.375000


In [90]:
chart = alt.Chart(df_piv.reset_index(), width=1500, height=1000, title=chart_title)\
.mark_line(color="white", point=alt.OverlayMarkDef(color="white"))\
.encode(
    x=alt.X(**x_cfgs,
    ),
    y=alt.Y(**y_cfgs,
    ),
    tooltip=[
        alt.Tooltip(shorthand="season_year", title="Year"),
        alt.Tooltip(shorthand="win_percentage", title="Win %"),
        alt.Tooltip(shorthand="W", title="Win"),
        alt.Tooltip(shorthand="L", title="Loss"),
    ]
)


rule = alt.Chart(df_piv.reset_index()).mark_rule(color='red').encode(
    y='mean(win_percentage):Q'
)

(chart + rule)\
.configure(background=primary_color)\
.configure_axis(gridColor="grey", gridDash=[6,4])\
.configure_axisLeft(**axisLeftKwargs)\
.configure_axisBottom(**axisBottomKwargs)\
.configure_title(color=secondary_color, fontSize=25)\
.configure_legend(**legendKwargs)

alt.LayerChart(...)